In [112]:
import pandas as pd
import numpy as np
from keras.layers import Input,LSTM,Dense,CuDNNLSTM,Bidirectional
from keras.models import Model
from keras.optimizers import Adam

In [2]:
data_path = '../data/cmn.txt'
df = pd.read_table(data_path,header=None)
df.columns=['inputs','targets']

#讲每句中文句首加上'\t'作为起始标志，句末加上'\n'作为终止标志
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')
df.head(5)

,inputs,targets
0,Hi.,\t嗨。\n
1,Hi.,\t你好。\n
2,Run.,\t你用跑的。\n
3,Wait!,\t等等！\n
4,Hello!,\t你好。\n


In [3]:
input_texts = df.inputs.values.tolist()#英文句子列表
target_texts = df.targets.values.tolist()#中文句子列表
print('input texts samples" {}'.format(len(input_texts)))

#确定中英文各自包含的字符。df.unique()直接取sum可将unique数组中的各个句子拼接成一个长句子
input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

print('last 5 character of input text: {}'.format(input_characters[-5:]))
print('last 5 character of target text: {}'.format(target_characters[-5:]))



input texts samples" 20294
last 5 character of input text: ['x', 'y', 'z', 'é', '’']
last 5 character of target text: ['，', '－', '：', '？', '𡡡']


## 每条句子经过对字母转换成one-hot编码后，生成了LSTM需要的三维输入[n_samples, timestamp, one-hot feature]

In [4]:
"""
NUM_SAMPLES，样本条数，这里是输入的句子条数
INPUT_LENGTH，输入数据的时刻t的长度，这里为最长的英文句子长度
OUTPUT_LENGTH，输出数据的时刻t的长度，这里为最长的中文句子长度
INPUT_FEATURE_LENGTH，每个时刻进入encoder的lstm单元的数据x t  xtx_t的维度，这里为英文中出现的字符数
OUTPUT_FEATURE_LENGTH，每个时刻进入decoder的lstm单元的数据x t  xtx_t的维度，这里为中文中出现的字符数
"""
NUM_SAMPLES = len(target_texts)
INPUT_LENGTH = max(df.inputs.apply(lambda x: len(x)))
OUTPUT_LENGTH = max(df.targets.apply(lambda x: len(x)))
INPUT_FEATURE_LENGTH = len(input_characters)
OUTPUT_FEATURE_LENGTH = len(target_characters)
print('NUM_SAMPLES: {}'.format(NUM_SAMPLES))
print('INPUT_LENGTH: {}'.format(INPUT_LENGTH))
print('OUTPUT_LENGTH: {}'.format(OUTPUT_LENGTH))
print('INPUT_FEATURE_LENGTH: {}'.format(INPUT_FEATURE_LENGTH))
print('OUTPUT_FEATURE_LENGTH: {}'.format(OUTPUT_FEATURE_LENGTH))

#encoder输入、decoder输入输出初始化为三维向量
encoder_input = np.zeros((NUM_SAMPLES,INPUT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))


input_dict = {char:index for index,char in enumerate(input_characters)}
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}




NUM_SAMPLES: 20294
INPUT_LENGTH: 163
OUTPUT_LENGTH: 46
INPUT_FEATURE_LENGTH: 75
OUTPUT_FEATURE_LENGTH: 3427


# 对句子进行字符级one-hot编码，将输入输出数据向量化：

# question 1: paddle at the end of a scentence ? vs paddle at the beginning of a scentence

In [5]:
#encoder的输入向量one-hot
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index,char_index,input_dict[char]] = 1

# question 2: decoder_input char_index starts from 0, should not be 1?

In [6]:
#decoder的输入输出向量one-hot，训练模型时decoder的输入要比输出晚一个时间步，这样才能对输出监督
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index,target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1,target_dict[char]] = 1.0

# MODELING

In [105]:
def create_model(n_input,n_output,n_units):
    """
    n_input: encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
    n_output: decoder的输入维度为中文字符数


    """
    #训练阶段
    encoder_input = Input(shape = (None, n_input))
    encoder = CuDNNLSTM(n_units, return_state=True)
    _,encoder_h,encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h,encoder_c]

    #decoder
    decoder_input = Input(shape = (None, n_output))
    decoder = CuDNNLSTM(n_units,return_sequences=True, return_state=True)
    decoder_output, _, _ = decoder(decoder_input,initial_state=encoder_state)
    decoder_dense = Dense(n_output,activation='softmax')
    decoder_output = decoder_dense(decoder_output)

    #生成的训练模型
    model = Model([encoder_input,decoder_input],decoder_output)

    #推理阶段，用于预测过程
    encoder_infer = Model(encoder_input,encoder_state)

    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   

    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
    decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
    decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)

    return model, encoder_infer, decoder_infer


In [164]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):#n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        state = [h,c]#本次状态做为下一次的初始状态继续传递
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':#预测到了终止符则停下来
            break
    return output


In [165]:
model, encoder_infer, decoder_infer = create_model(n_input=INPUT_FEATURE_LENGTH,
                                                   n_output=OUTPUT_FEATURE_LENGTH,
                                                   n_units=256)

In [166]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           (None, None, 75)     0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           (None, None, 3427)   0                                            
__________________________________________________________________________________________________
cu_dnnlstm_23 (CuDNNLSTM)       [(None, 256), (None, 340992      input_49[0][0]                   
__________________________________________________________________________________________________
cu_dnnlstm_24 (CuDNNLSTM)       [(None, None, 256),  3773440     input_50[0][0]                   
                                                                 cu_dnnlstm_23[0][1]              
          

In [167]:
model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.001),
        metrics=['accuracy'])


In [168]:
for j in range(500):
    print(j)
    model.fit(x=[encoder_input,decoder_output],y=decoder_input,batch_size=32)
    print(input_texts[i])
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(out)
    print('=============================================')

0
Epoch 1/1
20294/20294 [==============================] - 55s 3ms/step - loss: 1.1653 - acc: 0.0780
I can't see you.
	她人我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我是我
1
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 0.6644 - acc: 0.1577
I can't see you.
	請於瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪麗瑪
2
Epoch 1/1
20294/20294 [==============================] - 55s 3ms/step - loss: 0.3582 - acc: 0.2094
I can't see you.
	当于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T于T
3
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 0.2006 - acc: 0.2326
I can't see you.
		造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造
4
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 0.1170 - acc: 0.2438
I can't see you.
		造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造造
5
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 0.0706 - acc: 0.2498
I can't see you.
	列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列列
6
Epoch 1/

20294/20294 [==============================] - 53s 3ms/step - loss: 3.5513e-06 - acc: 0.2581
I can't see you.
	檸嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂
39
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 3.1884e-06 - acc: 0.2581
I can't see you.
	丑嬸丑嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸
40
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 2.9382e-06 - acc: 0.2581
I can't see you.
	檸嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂嬸剂
41
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 2.7491e-06 - acc: 0.2581
I can't see you.
	檸嬸檸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸嬸
42
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 2.5082e-06 - acc: 0.2581
I can't see you.
	檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸檸
43
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 2.3555e-06 - acc: 0.2581
I can't see you.
	宮窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長窩長

20294/20294 [==============================] - 53s 3ms/step - loss: 7.2532e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
77
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 7.0361e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐
78
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 6.9547e-07 - acc: 0.2581
I can't see you.
	哇窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐
79
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 6.8096e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
80
Epoch 1/1
20294/20294 [==============================] - 51s 3ms/step - loss: 6.6957e-07 - acc: 0.2581
I can't see you.
	丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑
81
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 6.5599e-07 - acc: 0.2581
I can't see you.
	丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑

20294/20294 [==============================] - 54s 3ms/step - loss: 4.0821e-07 - acc: 0.2581
I can't see you.
	哇拌脈莫脈莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎
115
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 4.0241e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐
116
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 3.9770e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
117
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 3.9342e-07 - acc: 0.2581
I can't see you.
	哇拌造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造
118
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 3.9009e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
119
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 3.8569e-07 - acc: 0.2581
I can't see you.
	哇窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩嚐窩

20294/20294 [==============================] - 53s 3ms/step - loss: 2.8749e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
153
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 2.8462e-07 - acc: 0.2581
I can't see you.
	哇拌造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造仿造
154
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 2.8261e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐
155
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 2.8046e-07 - acc: 0.2581
I can't see you.
	丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑丑
156
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 2.7875e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
157
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 2.7637e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤

...

20294/20294 [==============================] - 53s 3ms/step - loss: 1.2553e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
343
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.2512e-07 - acc: 0.2581
I can't see you.
	哇拌脈莫脈莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎
344
Epoch 1/1
20294/20294 [==============================] - 55s 3ms/step - loss: 1.2476e-07 - acc: 0.2581
I can't see you.
	哇拌脈莫脈莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎莫扎
345
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 1.2446e-07 - acc: 0.2581
I can't see you.
	哇趾造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造
346
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 1.2425e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
347
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.2380e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈

20294/20294 [==============================] - 54s 3ms/step - loss: 1.1434e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
381
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 1.1408e-07 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳趾紳这紳这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋
382
Epoch 1/1
20294/20294 [==============================] - 51s 3ms/step - loss: 1.1379e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐
383
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.1350e-07 - acc: 0.2581
I can't see you.
	哇趾造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造
384
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 1.1331e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
385
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.1304e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈

20294/20294 [==============================] - 52s 3ms/step - loss: 1.0538e-07 - acc: 0.2581
I can't see you.
	哇趾造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造
419
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.0514e-07 - acc: 0.2581
I can't see you.
	哇疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐疤嚐
420
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.0500e-07 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
421
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 1.0478e-07 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱坞莱
422
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 1.0459e-07 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳趾紳卵紳卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢
423
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 1.0433e-07 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐

20294/20294 [==============================] - 53s 3ms/step - loss: 9.8086e-08 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳趾紳趾紳这紳这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋
457
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 9.7855e-08 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳趾紳卵紳卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢卵廢
458
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 9.7737e-08 - acc: 0.2581
I can't see you.
	哇拌嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐嚐
459
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 9.7540e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
460
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 9.7368e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
461
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 9.7175e-08 - acc: 0.2581
I can't see you.
	哇趾紳趾紳趾紳趾紳这紳这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这栋这

20294/20294 [==============================] - 53s 3ms/step - loss: 9.1952e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
495
Epoch 1/1
20294/20294 [==============================] - 53s 3ms/step - loss: 9.1850e-08 - acc: 0.2581
I can't see you.
	哇趾造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造
496
Epoch 1/1
20294/20294 [==============================] - 52s 3ms/step - loss: 9.1647e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
497
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 9.1507e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
498
Epoch 1/1
20294/20294 [==============================] - 54s 3ms/step - loss: 9.1381e-08 - acc: 0.2581
I can't see you.
	哇脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈脈
499
Epoch 1/1
20294/20294 [==============================] - 55s 3ms/step - loss: 9.1215e-08 - acc: 0.2581
I can't see you.
	哇趾造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构造构

In [162]:
i = 1001
test = encoder_input[i:i+1,:,:]
out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
print(input_texts[i])
print(out)

I can't see you.
	塑價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價價


# out

In [67]:
train_lstm = model.get_layer('cu_dnnlstm_5')

In [72]:
print(train_lstm.weights[0])
train_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048403, -0.10057795,  0.09402207, ...,  0.06421671,
         0.04817443, -0.04124532],
       [ 0.02149276,  0.00541093,  0.11336602, ...,  0.12082654,
        -0.11954425, -0.08038449],
       [ 0.09851375, -0.12272102, -0.11307565, ..., -0.08199231,
        -0.02517138, -0.02650847],
       ...,
       [ 0.06733374,  0.02784984,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699327],
       [-0.05950085, -0.10947321,  0.0575029 , ..., -0.01508953,
        -0.04106094, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [77]:
infer_lstm = encoder_infer.get_layer('cu_dnnlstm_5')
print(infer_lstm.weights[0])
infer_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048403, -0.10057795,  0.09402207, ...,  0.06421671,
         0.04817443, -0.04124532],
       [ 0.02149276,  0.00541093,  0.11336602, ...,  0.12082654,
        -0.11954425, -0.08038449],
       [ 0.09851375, -0.12272102, -0.11307565, ..., -0.08199231,
        -0.02517138, -0.02650847],
       ...,
       [ 0.06733374,  0.02784984,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699327],
       [-0.05950085, -0.10947321,  0.0575029 , ..., -0.01508953,
        -0.04106094, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [79]:
infer_lstm = encoder_infer.get_layer('cu_dnnlstm_5')
print(infer_lstm.weights[0])
infer_lstm.get_weights()[0]

<tf.Variable 'cu_dnnlstm_5/kernel:0' shape=(75, 256) dtype=float32_ref>


array([[ 0.00048537, -0.10057795,  0.09402207, ...,  0.06423962,
         0.04818527, -0.04124521],
       [ 0.02149243,  0.00541081,  0.11336623, ...,  0.12081587,
        -0.11976563, -0.0803909 ],
       [ 0.09846777, -0.12273928, -0.11309011, ..., -0.08199231,
        -0.02517122, -0.02650827],
       ...,
       [ 0.06733384,  0.02784988,  0.02314961, ...,  0.09500158,
         0.12246257,  0.00699364],
       [-0.05950077, -0.10947321,  0.05750281, ..., -0.01508927,
        -0.04100636, -0.06283302],
       [-0.11229177,  0.06529395, -0.0673771 , ..., -0.0839394 ,
        -0.08219849,  0.09753321]], dtype=float32)

In [23]:
model.fit(x=[encoder_input,decoder_output],y=decoder_input,batch_size=512)

Epoch 1/1
20294/20294 [==============================] - 40s 2ms/step - loss: 1.9426 - acc: 0.0410


In [ ]:
model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

In [ ]:
for i in range(1000,1210):
    test = encoder_input[i:i+1,:,:]#i:i+1保持数组是三维
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(input_texts[i])
    print(out)
